# How much do you pay, and how much you should pay
Just answering 3 questions, I'll give you the actual market price of your real state and I will tell you how mucho you sould pay for it, supposing that the flat is in a standard condicitions, like the most flats in Barcelona.
Also I will give you the maximun and minimun price that you should pay.

Maximinun price: The price in case the real state is in really good conditions.

Minimun price: The price in case the real state is in bad conditions.

In [23]:
RealStatePrice()

Where do you live?
With your postal code is okey.  08001

And is it a flat, apartment, attic or a study?  attic

Do you know how many m2 does it have?
If not just write I dont know (idk)
I will give you the price per m2.
60

This is the price in your postal code area, 8001


,€
Market price,1386.0
Lloguer index price,798.0
Max price,857.0
Min price,741.0


### Import all files

In [1]:
import pandas as pd
import numpy as np
import time
import re

In [2]:
#Import postal codes
CP = pd.read_csv('csv/postal_codes.csv')
CP = CP.drop(['Unnamed: 0'],axis=1)

In [3]:
#Import fotocasa dataset
fotocasa = pd.read_csv('csv/fotocasa_clean.csv')
fotocasa_pivot = fotocasa.pivot_table(index = ['neighborhood','real_state'],
                                      values = ['square_meters_price'], aggfunc = ['mean']).reset_index()

In [4]:
#Import lloguer dataset
lloguer = pd.read_csv('csv/lloguer.csv')
lloguer = lloguer.drop(['FID','ID','DISTRICTE','CONTRAC','CONT1000H','VARLLOG','LLOGTRIM','GEOMETRY'],axis=1)
lloguer['DISTR_BCN']= lloguer['DISTR_BCN'].map({'Eixample':'Eixample','Horta-Guinardó':'Horta- Guinardo', 
                              'Sarrià-Sant Gervasi': 'Sarria-Sant Gervasi','Ciutat Vella':'Ciutat Vella','Gràcia':'Gràcia',
                              'Sant Martí':'Sant Martí','Sants-Montjuïc':'Sants-Montjuïc',
                             'Les Corts':'Les Corts','Sant Andreu':'Sant Andreu','Nou Barris':'Nou Barris'})

### Cleaning the files

In [5]:
lloguer['max_price'] = lloguer['TRAMLLOG'].str.split('<').str[-1]
lloguer['max_price'] = list(map(lambda x:re.findall('\d+',x),lloguer['max_price'] ))
lloguer['min_price'] = lloguer['TRAMLLOG'].str.split('<').str[0]
lloguer['min_price'] = list(map(lambda x:re.findall('[\d\s]+i',x),lloguer['min_price'] ))

In [6]:
def fill_space(row):
    if row == []:
        return ['0']
    else:
        return row

In [7]:
def list_clean(row):
    return row[0]

In [8]:
lloguer['min_price'] = lloguer['min_price'].apply(fill_space)
lloguer['max_price'] = lloguer['max_price'].apply(list_clean)
lloguer['min_price'] = lloguer['min_price'].apply(list_clean)
lloguer['min_price'] = list(map(lambda x:re.findall('\d+',x),lloguer['min_price'] ))
lloguer['min_price'] = lloguer['min_price'].apply(list_clean)

In [9]:
for row in range(len(lloguer['max_price'])):
    if lloguer.loc[row,'min_price']== '0':
        lloguer.loc[row,'min_price'] = lloguer.loc[row,'max_price'] 
        lloguer.loc[row,'max_price'] = None

In [10]:
lloguer['max_price'] = pd.to_numeric(lloguer['max_price'])
lloguer['min_price'] = pd.to_numeric(lloguer['min_price'])
lloguer['max_price_m2'] = lloguer['max_price']/lloguer['SUPMITJ']
lloguer['min_price_m2'] = lloguer['min_price']/lloguer['SUPMITJ']
lloguer['PREUM2'] = lloguer['LLOGUER']/lloguer['SUPMITJ']

In [11]:
lloguer['max_price_m2'] = lloguer['PREUM2']-((int(89)-lloguer['PREUM2'])/int(73.63))*(int(1)*int(0.7))+ 1

In [12]:
lloguer

,DISTR_BCN,VARCONTR,TRAMLLOG,LLOGUER,SUPMITJ,PREUM2,PREUM2TRIM,max_price,min_price,max_price_m2,min_price_m2
0,Ciutat Vella,-23.811620,> 850 i <= 1000 euros/mes,913.664714,68.796053,13.280772,13.729155,1000.0,850,14.280772,12.355360
1,Eixample,-22.967073,> 1000 euros/mes,1075.436849,80.075509,13.430284,13.767873,NaN,1000,14.430284,12.488213
2,Sants-Montjuïc,-19.210123,> 750 i <= 850 euros/mes,846.878075,64.607409,13.108064,13.374586,850.0,750,14.108064,11.608576
3,Les Corts,-19.913043,> 1000 euros/mes,1112.496379,79.295508,14.029753,14.272469,NaN,1000,15.029753,12.611055
4,Sarria-Sant Gervasi,-22.001871,> 1000 euros/mes,1305.726906,90.865045,14.369958,15.100435,NaN,1000,15.369958,11.005332
5,Gràcia,-23.621738,> 850 i <= 1000 euros/mes,950.362510,69.741999,13.626832,14.202070,1000.0,850,14.626832,12.187778
6,Horta- Guinardo,-23.812785,> 750 i <= 850 euros/mes,799.317279,65.082817,12.281541,12.386828,850.0,750,13.281541,11.523779
7,Nou Barris,-20.907114,<= 750 euros/mes,705.505805,62.181034,11.345997,11.491785,NaN,750,12.345997,12.061556
8,Sant Andreu,-17.435207,> 750 i <= 850 euros/mes,797.823836,67.134749,11.883918,12.188302,850.0,750,12.883918,11.171562
9,Sant Martí,-17.403460,> 850 i <= 1000 euros/mes,929.907692,70.261047,13.235039,13.392316,1000.0,850,14.235039,12.097742


### Fuction

In [17]:
def RealStatePrice():
    adress = input("Where do you live?\nWith your postal code is okey.  ")
    adress = int(adress)
    neighborhood = CP[CP['CP']==adress].iloc[0,1]
    time.sleep(0.6)
    
    real_state = input('\nAnd is it a flat, apartment, attic or a study?  ')
    time.sleep(0.6)
    
    m2 = input('\nDo you know how many m2 does it have?\nIf not just write I dont know (idk)'
               '\nI will give you the price per m2.\n')
    time.sleep(0.6)
    
    market_price = [round(fotocasa_pivot[(fotocasa_pivot['neighborhood']==neighborhood) & 
                         (fotocasa_pivot['real_state']==real_state)].iloc[0,2],1)]
    lloguer_price = [round(lloguer[lloguer['DISTR_BCN']==neighborhood].iloc[0,5],1)]
    
    
    if m2.isnumeric():
        m2= int(m2)
        state_market_price = [round(market_price[0]*m2,0)]
        state_lloguer_price = [round(lloguer_price[0]*m2,0)]
        
        max_m2_price = [lloguer[lloguer['DISTR_BCN']==neighborhood].iloc[0,9]]
        min_m2_price = [lloguer[lloguer['DISTR_BCN']==neighborhood].iloc[0,10]]
        
        max_pay = [round(max_m2_price[0]*m2,0)]
        min_pay = [round(min_m2_price[0]*m2,0)]
        
        time.sleep(0.6)
        
        print('\nThis is the price in your postal code area,',adress)
        state_price = pd.DataFrame({'Market price':state_market_price,
                              'Lloguer index price':state_lloguer_price,'Max price':max_pay,
                           'Min price':min_pay}).T
        state_price.columns = ['€']
        display(state_price)
        
    else:
        print('\nYou can check it here,\nhttps://www1.sedecatastro.gob.es/Cartografia/mapa.aspx?buscar=S'
        '\njust introducing your address.\n')
        time.sleep(2)
        
        max_pay_m2 = [lloguer[lloguer['DISTR_BCN']==neighborhood].iloc[0,9]]
        min_pay_m2 = [lloguer[lloguer['DISTR_BCN']==neighborhood].iloc[0,10]]
        
        print('This is the price in your postal code area,',adress)
        m2_price = pd.DataFrame({'Market price':market_price,
                              'Lloguer index price':lloguer_price,
                                 'Max price':max_pay_m2,
                           'Min price':min_pay_m2}).T
        
        m2_price.columns = ['€/m2']
        display(m2_price)